In [1]:
import numpy as np
import random
import math
from IPython.display import display, clear_output
from sklearn.utils import shuffle as shuffle_data

random.seed(0)

# Layer represents a MLP Layer
# It has two main properties:
#      - a weigth matrix containing the weights of the layer's neurons. Each line represents a neuron and 
#        the columns represent its corresponding weights
#      - a bias vector, containing the neurons's bias
# Since during the backpropagation we need to compute the weights variation using the old ones, the 
# updated_weights and updated_bias properties store the new values until the update method is called
class Layer:
    # Create a new Layer with 'size' neurons, each one linked to 'inputs_size' inputs
    def __init__(self, size, inputs_size):
        self.size = size
        self.inputs_size = inputs_size
        self.weights = np.array([[random.uniform(-0.1, 0.1) for j in range(inputs_size)] for i in range(size)])
        self.bias = np.array([random.uniform(-0.1,0.1) for i in range(size)])
        
        self.d_weights_current = np.zeros((size, inputs_size))
        self.d_bias_current = np.zeros(size)
        self.d_weights_old = np.zeros((size, inputs_size))
        self.d_bias_old = np.zeros(size)
    
    # update updates the weights and bias matrices with the values stored in the updated ones
    def update(self, eta, alpha):
        #self.weights = np.copy(self.updated_weights)
        #self.bias = np.copy(self.updated_bias)
        
        self.weights = self.weights + eta*self.d_weights_current + alpha*self.d_weights_old 
        self.bias = self.bias + eta*self.d_bias_current + alpha*self.d_bias_old
        
        self.d_weights_old = self.d_weights_current
        self.d_bias_old = self.d_bias_current
        
    # description prints a layer description
    def description(self):
        print("Layer Info")
        print("Weights: \n", self.weights)
        print("Bias: \n ", self.bias)

def logistic(x):
    return 1.0/(1.0+ math.exp(-x))

logistic_vec = np.vectorize(logistic)

def logistic_derivate(x):
    return x*(1.0-x)

In [2]:
class MLP:
    # MLP creation. One might pass the MLP layers as parameters or add them later using the add_layer method.
    # The classification parameter defines if the MLP will be used for a classification or regression problem
    def __init__(self, *layers, classifier=True):
        self.classifier = classifier
        if classifier:
            # Map each class label to a vector with a single 1
            # Ex: Class 0 -> [1,0]
            #     Class 1 -> [0,1]
            self.class_mapping = dict()  
            # Unmap each class vector to the corresponding class label
            # Ex: [1,0] -> Class 0 
            #     [0,1] -> Class 1
            self.class_unmapping = dict()
            
        self.layers = list()
        for layer in layers:
            self.add_layer(layer)
    
    # Shortcut to create a classifier MLP
    @classmethod
    def MLPClassifier(cls, *layers):
        return cls(classifier=True, *layers)   
    
    # Shortcut to create a regressor MLP
    @classmethod
    def MLPRegressor(cls, *layers):
        return cls(classifier=False, *layers)
    
    # add_layer adds a new layer on the MLP. It verifies whether or not the new layer is compatible with the MLP
    def add_layer(self, layer):
        # If there's already a layer in the MLP, verify if the new layer is compatible
        if len(self.layers) > 0:
            if layer.inputs_size != self.layers[-1].size:
                print("The new layer is incompatible with the MLP")
                print("Please, use a layer where each neuron has the same amount of inputs as the number" \
                     "of neurons in the MLP last layer")
        
        self.layers.append(layer)
    
    # description prints the info about the MLP layers
    def description(self):
        print("MLP Classifier?: ", self.classifier)
        print("-------------------------")
        print("MLP Info:")
        for layer, i in zip(self.layers, range(len(self.layers))):
            print("--- Layer: %d ---" % i)
            layer.description()
            
    # __get_class_mapping gets the class labels in the classes list and builds the mapping dicionaries
    # class_mapping and class_unmapping
    def __get_class_mapping(self, classes):
        class_labels = np.unique(classes)
        
        for c in range(len(class_labels)):
            class_label = class_labels[c]
            class_vector = np.zeros(len(class_labels))
            class_vector[c] = 1
    
            self.class_mapping[class_label] = class_vector
            
            # We can't use a list as a hash key. So transform it into a tuple
            self.class_unmapping[tuple(class_vector)] = class_label
        
    # __convert_class_labels_to_vectors converts a list with class labels to a list with 
    # vectors that maps each class label
    def __convert_class_labels_to_vectors(self, class_labels):
        return [self.class_mapping[c] for c in class_labels]
    
    # __convert_class_vectors_to_labels converts a list with class vectors to a list with 
    # the corresponding class labels
    def __convert_class_vectors_to_labels(self, class_vectors):
        return [self.class_unmapping[tuple(class_vector)] for class_vector in class_vectors]
        
        
    # fast_forward computes the ouput for a given input vector
    def fast_forward(self,input_v):
        # We need to store each layer input in order to perform the backpropagation
        self.inputs = list()
    
        # The input is applied in a layer weights matrix and the bias is added in the result
        # Then, the logistic function is applied to each layer neuron result
        # For a layer, we have a final output vector where each component i represents the output
        # of the neuron i
        for layer in self.layers:
            self.inputs.append(input_v)
            output = logistic_vec(layer.weights @ input_v + layer.bias)
            
            # The output of the current layer is the input of the next one
            input_v = output
        
        return output
    
    # train trains the MLP using the examples passed in the samples parameter
    # The expected output for each example must be passed in the classes parameter;
    # eta represents the MLP learning rate;
    # tol represents the error tolerance. The MLP is trained until the cumulative squared error for all example
    #     is less than the tol value
    # print_status prints the output for each example during the training phase
    def train(self, samples, classes, eta=0.5, alpha=0, tol=1e-2, epoch_max=2000, 
              print_status=False, shuffle=True):
        # Map the class labels to output vectors if it's a classification problem
        if self.classifier:
            self.__get_class_mapping(classes)
            classes = self.__convert_class_labels_to_vectors(classes)
                
        error = tol
        new_error = 3*tol
        epoch = 0
        
        # The training stops when the max number of epochs is reached or the Kramer and Sangiovanni-Vicentelly
        # criteria is valid. According to it, we can consider that the BP converged when the average mean squared
        # error is less than a given tolerance
        while (abs(new_error - error) > tol and epoch < epoch_max):
            epoch += 1
            error = 0
            new_error = 0
            
            # Suffles samples to avoid saturation if training with samples beloging to the same class
            # one after another
            if shuffle:
                samples, classes = shuffle_data(samples, classes)
            
            for input_v, t in zip(samples, classes):  
                # ---- Compute the output for the given input vector ----
                output = self.fast_forward(input_v)
                
                # Compute the mean squared error before the backpropagation
                error_sample = pow((np.array(t)-np.array(output)),2)
                # We need to sum the error of each component when the output is a vector
                error += sum(error_sample)/len(samples)
                
                if (print_status == True):
                    print("\ttraining example: %s from class %s" % (input_v, t), end = " ")
                    print("y = ", output)
     
                # ---- Backpropagation ----
                # Compute the new weights of each layer
                # Remark: the udpated weights are stored as a layer property and the layer is updated once 
                # the backpropagation is finished
                # It's necessary to do so in order to compute the delta value for the inner layers. We need 
                # to use the weights that caused the error to compute the delta instead of the updated weights
                for l in reversed(range(len(self.layers))): # Traverse the layers in reversed order
                    layer = self.layers[l]
             
                    deltas = list()
                    # Compute the delta for each layer neuron n
                    for n in range(len(layer.weights)):
                        # Last Layer
                        if l == (len(self.layers)-1):
                            delta = (t[n]-output[n])*logistic_derivate(output[n])
                            
                        # Inner Layer
                        else:
                            # output of the current layer is the input of the next one
                            neuron_output = self.inputs[l+1][n]
                            # weights of each neuron output
                            errors_weights = self.layers[l+1].weights[:,n]
                            
                            delta = np.dot(delta_next_layer,errors_weights)*logistic_derivate(neuron_output)
                              
                        # Computes the weights and bias variation for the neuron n
                        for w in range(len(layer.weights[n])):
                            layer.d_weights_current[n][w] = delta*self.inputs[l][w]
                        layer.d_bias_current[n] = delta*1 # bias input = 1
                        
                        #for w in range(len(layer.weights[n])):
                        #    layer.updated_weights[n][w] = layer.weights[n][w] + eta*delta*self.inputs[l][w]
                        #layer.updated_bias[n] = layer.bias[n] + (eta*delta*1) # bias input = 1

                        # Store the neuron delta
                        deltas.append(delta)
                    
                    # The neurons' delta of the current layer will be used to compute the deltas of the 
                    # next inner layer
                    delta_next_layer = np.array(deltas)
                     
                # Once the backpropagation is finished for the current example, update all the weigths and bias
                for layer in self.layers:
                    layer.update(eta, alpha)
                    
                # Compute the new error mean squared error
                output = self.fast_forward(input_v)
                error_sample = pow((np.array(t)-np.array(output)),2)
                #print("error sample: ",error_sample)
                new_error += sum(error_sample)/len(samples)
            
            # End of a epoch
            if epoch%1 == 0: # Print status only after each 100 iterations 
                clear_output(wait=True)
                display("End of epoch " + str(epoch) + ". Total Error = " + str(new_error))
        
        # End of training         
        clear_output(wait=True)
        display("End of epoch " + str(epoch) + ". Total Error = " + str(new_error))
        
    # predicts gets a list of input samples and returns a list with the predicted outputs
    def predict(self, samples):
        outputs = list()
        for input_v in samples:
            probs = self.fast_forward(input_v)
            
            if self.classifier:
                class_pos = np.argmax(probs)
                output = np.zeros(len(probs))
                output[class_pos] = 1
            
                #outputs.append(self.class_unmapping[tuple(output)])
                outputs.append(output)
                
            else:
                outputs.append(probs)
    
        if self.classifier:
            return self.__convert_class_vectors_to_labels(outputs)
        
        else:
            return outputs

# Pre Processing Data

In [3]:
# normalize data transforms data in order to all points have mean 0 and variance 1
def normalize_data(data):
    normalized_columns = list()
    for c in range(len(data[0])):
        col = data[:,c]
        normalized_columns.append((col - np.mean(col))/np.std(col))

    return np.array(normalized_columns).T

In [4]:
# scale_data transforms data in order to all points be in the interval [0,1]
def scale_data(data):
    normalized_columns = list()
    for c in range(len(data[0])):
        col = data[:,c]
        normalized_columns.append((col-np.min(col))/(np.max(col)-np.min(col)))

    return np.array(normalized_columns).T

# Evaluation

In [150]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import pandas as pd

def evaluate(real_classes, predicted_classes):
    acc = accuracy_score(real_classes, predicted_classes)
    print("Accuracy: %.2f%%" % (acc*100))
    
    class_labels = np.sort(np.unique(real_classes))
    cm = confusion_matrix(real_classes, predicted_classes, labels=class_labels)
    df = pd.DataFrame(cm)
    df.columns = class_labels
    df.index = class_labels
    
    # Acurracy per class
    accs = list()
    for c in range(len(cm)):
        accs.append(cm[c,c]/sum(cm[c,:]))
    df["Accuracy"] = accs
        
    print("Confusion Matrix and Accuracy per class:")
    print(df)
        
    avg_acc = np.average(accs)
    print("Average accuracy per class: %.2f%%" % (avg_acc*100))
    return acc, avg_acc

# Classification

Para o problema de classificação, vamos pegar o conjunto de dados fornecidos e vamos normalizá-los de forma que cada valor de feature esteja no intervalo (0,1). Isso será feito para evitar a saturação da saída dos neurônios e melhorar a convergência do algoritmo de aprendizagem.

A seguir, tomaremos um conjunto de treinamento consistindo em 70% da base original. Iremos avaliar o impacto da arquitetura da rede assim como da variação de parâmetros de aprendizado no valor de acurácia obtido na classificação do conjunto de teste. 

Iremos calcular a acurácia total e a acurácia por classe. Uma vez que não estamos impondo penalidades diferentes para erros cometidos em determinadas classes, vamos considerar como melhor arquitetura aquela que fornece maior valor para a acurácia total, embora isso possa não refletir em uma acurácia por classe elevada! Mais tarde, iremos tratar melhor o balanceamento entre as classes a fim de que o aprendizado da rede seja adequado para todas elas.

## Original DataSet

In [5]:
import pandas as pd

df = pd.read_csv('winequality-red.csv')
df.head(5)

,Unnamed: 0,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,category
0,0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,Mid
1,1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,Mid
2,2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,Mid
3,3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,Mid
4,4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,Mid


In [6]:
# Separação dos Dados
inputs = df[df.columns[1:-1]].values
classes = df[df.columns[-1]].values

print(inputs[0:2,:])
print(classes)

[[ 7.4     0.7     0.      1.9     0.076  11.     34.      0.9978  3.51
   0.56    9.4   ]
 [ 7.8     0.88    0.      2.6     0.098  25.     67.      0.9968  3.2
   0.68    9.8   ]]
['Mid' 'Mid' 'Mid' ... 'Mid' 'Mid' 'Mid']


In [8]:
unique, counts = np.unique(classes, return_counts=True)
print(unique)
print(counts)

['Bad' 'Good' 'Mid']
[  63  217 1319]


In [9]:
scaled_inputs = scale_data(inputs)

### Split - Training - Test

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(scaled_inputs, classes, test_size=0.3, 
                                                    stratify=classes,random_state=42)

# Testing different network architectures

### 1 layer - 5 Neurons

In [117]:
# Training
N1 = 5
N = len(X_train[0])
n_classes = len(np.unique(y_train))

random.seed(0)
mlp = MLP.MLPClassifier(Layer(N1, N), Layer(n_classes, N1))
mlp.train(X_train, y_train, eta=0.5, alpha=0.5, tol=1e-4, epoch_max=400, print_status=False, shuffle=True)

'End of epoch 400. Total Error = 0.17529597439174865'

In [116]:
predicted = mlp.predict(X_test)
evaluate(y_test, predicted)

Accuracy: 83.96%
Confusion Matrix and Accuracy per class:
      Bad  Good  Mid  Accuracy
Bad     3     0   16  0.157895
Good    0    21   44  0.323077
Mid     8     9  379  0.957071
Average accuracy per class: 47.93%


### 1 Layer - 11 Neurons

In [122]:
N1 = 11
N = len(X_train[0])
n_classes = len(np.unique(y_train))

random.seed(0)
mlp = MLP.MLPClassifier(Layer(N1, N), Layer(n_classes, N1))
mlp.train(X_train, y_train, eta=0.5, alpha=0.5, tol=1e-4, epoch_max=400, print_status=False, shuffle=True)

'End of epoch 400. Total Error = 0.1733605344785225'

In [123]:
predicted = mlp.predict(X_test)
evaluate(y_test, predicted)

Accuracy: 82.50%
Confusion Matrix and Accuracy per class:
      Bad  Good  Mid  Accuracy
Bad     0     0   19  0.000000
Good    0    13   52  0.200000
Mid     2    11  383  0.967172
Average accuracy per class: 38.91%


### 1 Layer - 22 Neurons

In [124]:
N1 = 22
N = len(X_train[0])
n_classes = len(np.unique(y_train))

random.seed(0)
mlp = MLP.MLPClassifier(Layer(N1, N), Layer(n_classes, N1))
mlp.train(X_train, y_train, eta=0.5, alpha=0.5, tol=1e-4, epoch_max=400, print_status=False, shuffle=True)

'End of epoch 400. Total Error = 0.15788458253020296'

In [125]:
predicted = mlp.predict(X_test)
evaluate(y_test, predicted)

Accuracy: 83.12%
Confusion Matrix and Accuracy per class:
      Bad  Good  Mid  Accuracy
Bad     1     0   18  0.052632
Good    0    24   41  0.369231
Mid     6    16  374  0.944444
Average accuracy per class: 45.54%


### 2 layers: 5 Neurons - 5 Neurons

In [126]:
N1 = 5
N2 = 5
N = len(X_train[0])
n_classes = len(np.unique(y_train))

random.seed(0)
mlp = MLP.MLPClassifier(Layer(N1, N), Layer(N2, N1), Layer(n_classes, N2))
mlp.train(X_train, y_train, eta=0.5, alpha=0.5, tol=1e-4, epoch_max=400, print_status=False, shuffle=True)

'End of epoch 400. Total Error = 0.18066216404897734'

In [127]:
predicted = mlp.predict(X_test)
evaluate(y_test, predicted)

Accuracy: 83.33%
Confusion Matrix and Accuracy per class:
      Bad  Good  Mid  Accuracy
Bad     1     0   18  0.052632
Good    0    24   41  0.369231
Mid     6    15  375  0.946970
Average accuracy per class: 45.63%


### 2 layers: 5 Neurons - 11 Neurons

In [128]:
N1 = 5
N2 = 11
N = len(X_train[0])
n_classes = len(np.unique(y_train))

random.seed(0)
mlp = MLP.MLPClassifier(Layer(N1, N), Layer(N2, N1), Layer(n_classes, N2))
mlp.train(X_train, y_train, eta=0.5, alpha=0.5, tol=1e-4, epoch_max=400, print_status=False, shuffle=True)

'End of epoch 400. Total Error = 0.17173439984447603'

In [129]:
predicted = mlp.predict(X_test)
evaluate(y_test, predicted)

Accuracy: 83.75%
Confusion Matrix and Accuracy per class:
      Bad  Good  Mid  Accuracy
Bad     2     0   17  0.105263
Good    0    34   31  0.523077
Mid     3    27  366  0.924242
Average accuracy per class: 51.75%


### 2 layers: 5 Neurons - 22 Neurons

In [130]:
N1 = 5
N2 = 22
N = len(X_train[0])
n_classes = len(np.unique(y_train))

random.seed(0)
mlp = MLP.MLPClassifier(Layer(N1, N), Layer(N2, N1), Layer(n_classes, N2))
mlp.train(X_train, y_train, eta=0.5, alpha=0.5, tol=1e-4, epoch_max=400, print_status=False, shuffle=True)

'End of epoch 400. Total Error = 0.16735402370755087'

In [131]:
predicted = mlp.predict(X_test)
evaluate(y_test, predicted)

Accuracy: 82.92%
Confusion Matrix and Accuracy per class:
      Bad  Good  Mid  Accuracy
Bad     2     0   17  0.105263
Good    0    41   24  0.630769
Mid     3    38  355  0.896465
Average accuracy per class: 54.42%


### 2 layers: 11Neurons - 5 Neurons

In [132]:
N1 = 11
N2 = 5
N = len(X_train[0])
n_classes = len(np.unique(y_train))

random.seed(0)
mlp = MLP.MLPClassifier(Layer(N1, N), Layer(N2, N1), Layer(n_classes, N2))
mlp.train(X_train, y_train, eta=0.5, alpha=0.5, tol=1e-4, epoch_max=400, print_status=False, shuffle=True)

'End of epoch 400. Total Error = 0.17190566880333363'

In [133]:
predicted = mlp.predict(X_test)
evaluate(y_test, predicted)

Accuracy: 82.92%
Confusion Matrix and Accuracy per class:
      Bad  Good  Mid  Accuracy
Bad     1     0   18  0.052632
Good    0    36   29  0.553846
Mid     3    32  361  0.911616
Average accuracy per class: 50.60%


### 2 layers: 11Neurons - 11 Neurons

In [135]:
N1 = 11
N2 = 11
N = len(X_train[0])
n_classes = len(np.unique(y_train))

random.seed(0)
mlp = MLP.MLPClassifier(Layer(N1, N), Layer(N2, N1), Layer(n_classes, N2))
mlp.train(X_train, y_train, eta=0.5, alpha=0.5, tol=1e-4, epoch_max=400, print_status=False, shuffle=True)

'End of epoch 400. Total Error = 0.15694439779030575'

In [136]:
predicted = mlp.predict(X_test)
evaluate(y_test, predicted)

Accuracy: 84.58%
Confusion Matrix and Accuracy per class:
      Bad  Good  Mid  Accuracy
Bad     1     0   18  0.052632
Good    0    28   37  0.430769
Mid     2    17  377  0.952020
Average accuracy per class: 47.85%


### 2 layers: 11Neurons - 22 Neurons

In [137]:
N1 = 11
N2 = 22
N = len(X_train[0])
n_classes = len(np.unique(y_train))

random.seed(0)
mlp = MLP.MLPClassifier(Layer(N1, N), Layer(N2, N1), Layer(n_classes, N2))
mlp.train(X_train, y_train, eta=0.5, alpha=0.5, tol=1e-4, epoch_max=400, print_status=False, shuffle=True)

'End of epoch 400. Total Error = 0.1755472399373921'

In [138]:
predicted = mlp.predict(X_test)
evaluate(y_test, predicted)

Accuracy: 83.96%
Confusion Matrix and Accuracy per class:
      Bad  Good  Mid  Accuracy
Bad     1     0   18  0.052632
Good    0    26   39  0.400000
Mid     3    17  376  0.949495
Average accuracy per class: 46.74%


### 2 layers: 22Neurons - 5 Neurons

In [141]:
N1 = 22
N2 = 5
N = len(X_train[0])
n_classes = len(np.unique(y_train))

random.seed(0)
mlp = MLP.MLPClassifier(Layer(N1, N), Layer(N2, N1), Layer(n_classes, N2))
mlp.train(X_train, y_train, eta=0.5, alpha=0.5, tol=1e-4, epoch_max=400, print_status=False, shuffle=True)

'End of epoch 400. Total Error = 0.15953557131697943'

In [142]:
predicted = mlp.predict(X_test)
evaluate(y_test, predicted)

Accuracy: 82.29%
Confusion Matrix and Accuracy per class:
      Bad  Good  Mid  Accuracy
Bad     3     0   16  0.157895
Good    0    37   28  0.569231
Mid     7    34  355  0.896465
Average accuracy per class: 54.12%


0.541196717512507

### 2 layers: 22Neurons - 11 Neurons

In [143]:
N1 = 22
N2 = 11
N = len(X_train[0])
n_classes = len(np.unique(y_train))

random.seed(0)
mlp = MLP.MLPClassifier(Layer(N1, N), Layer(N2, N1), Layer(n_classes, N2))
mlp.train(X_train, y_train, eta=0.5, alpha=0.5, tol=1e-4, epoch_max=400, print_status=False, shuffle=True)

'End of epoch 400. Total Error = 0.15513180908747362'

In [144]:
predicted = mlp.predict(X_test)
evaluate(y_test, predicted)

Accuracy: 84.17%
Confusion Matrix and Accuracy per class:
      Bad  Good  Mid  Accuracy
Bad     1     0   18  0.052632
Good    0    30   35  0.461538
Mid     6    17  373  0.941919
Average accuracy per class: 48.54%


0.4853630774683406

### 2 layers: 22Neurons - 22 Neurons

In [145]:
N1 = 22
N2 = 22
N = len(X_train[0])
n_classes = len(np.unique(y_train))

random.seed(0)
mlp = MLP.MLPClassifier(Layer(N1, N), Layer(N2, N1), Layer(n_classes, N2))
mlp.train(X_train, y_train, eta=0.5, alpha=0.5, tol=1e-4, epoch_max=400, print_status=False, shuffle=True)

'End of epoch 400. Total Error = 0.15342987352435103'

In [146]:
predicted = mlp.predict(X_test)
evaluate(y_test, predicted)

Accuracy: 83.33%
Confusion Matrix and Accuracy per class:
      Bad  Good  Mid  Accuracy
Bad     1     2   16  0.052632
Good    0    45   20  0.692308
Mid     2    40  354  0.893939
Average accuracy per class: 54.63%


0.5462928883981516

# Number of epochs used during training

Vamos analisar como o número de epochs usadas durante o treinamento interfere na acurácia obtida no conjunto de testes! Para issom, vamos utilizar a arquitetura de rede que obteve melhor desempenho:



In [147]:
accs = dict()
best_N1 = 11
best_N2 = 11

# 200 epochs

In [151]:
epochs = 200

random.seed(0)
best_layers = [Layer(best_N1, N), Layer(best_N2, best_N1), Layer(n_classes, best_N2)]
mlp = MLP.MLPClassifier(*best_layers)
mlp.train(X_train, y_train, eta=0.5, alpha=0.5, tol=1e-4, epoch_max=epochs, print_status=False, shuffle=True)

predicted = mlp.predict(X_test)
accs[epochs] = evaluate(y_test, predicted)

'End of epoch 200. Total Error = 0.1954606515584208'

Accuracy: 83.12%
Confusion Matrix and Accuracy per class:
      Bad  Good  Mid  Accuracy
Bad     1     1   17  0.052632
Good    0    19   46  0.292308
Mid     1    16  379  0.957071
Average accuracy per class: 43.40%


# 400 epochs

In [153]:
epochs = 400

random.seed(0)
best_layers = [Layer(best_N1, N), Layer(best_N2, best_N1), Layer(n_classes, best_N2)]
mlp = MLP.MLPClassifier(*best_layers)
mlp.train(X_train, y_train, eta=0.5, alpha=0.5, tol=1e-4, epoch_max=epochs, print_status=False, shuffle=True)

predicted = mlp.predict(X_test)
accs[epochs] = evaluate(y_test, predicted)

'End of epoch 400. Total Error = 0.15824168927950283'

Accuracy: 84.38%
Confusion Matrix and Accuracy per class:
      Bad  Good  Mid  Accuracy
Bad     1     0   18  0.052632
Good    0    34   31  0.523077
Mid     7    19  370  0.934343
Average accuracy per class: 50.34%


# 800 epochs

In [154]:
epochs = 800

random.seed(0)
best_layers = [Layer(best_N1, N), Layer(best_N2, best_N1), Layer(n_classes, best_N2)]
mlp = MLP.MLPClassifier(*best_layers)
mlp.train(X_train, y_train, eta=0.5, alpha=0.5, tol=1e-4, epoch_max=epochs, print_status=False, shuffle=True)

predicted = mlp.predict(X_test)
accs[epochs] = evaluate(y_test, predicted)

'End of epoch 800. Total Error = 0.11989796910609325'

Accuracy: 83.96%
Confusion Matrix and Accuracy per class:
      Bad  Good  Mid  Accuracy
Bad     2     0   17  0.105263
Good    0    35   30  0.538462
Mid     7    23  366  0.924242
Average accuracy per class: 52.27%


# 1000 epochs

In [155]:
epochs = 1000

random.seed(0)
best_layers = [Layer(best_N1, N), Layer(best_N2, best_N1), Layer(n_classes, best_N2)]
mlp = MLP.MLPClassifier(*best_layers)
mlp.train(X_train, y_train, eta=0.5, alpha=0.5, tol=1e-4, epoch_max=epochs, print_status=False, shuffle=True)

predicted = mlp.predict(X_test)
accs[epochs] = evaluate(y_test, predicted)

'End of epoch 1000. Total Error = 0.11785875370497366'

Accuracy: 82.50%
Confusion Matrix and Accuracy per class:
      Bad  Good  Mid  Accuracy
Bad     2     2   15  0.105263
Good    0    24   41  0.369231
Mid     5    21  370  0.934343
Average accuracy per class: 46.96%


# 2000 epochs

In [156]:
epochs = 2000

random.seed(0)
best_layers = [Layer(best_N1, N), Layer(best_N2, best_N1), Layer(n_classes, best_N2)]
mlp = MLP.MLPClassifier(*best_layers)
mlp.train(X_train, y_train, eta=0.5, alpha=0.5, tol=1e-4, epoch_max=epochs, print_status=False, shuffle=True)

predicted = mlp.predict(X_test)
accs[epochs] = evaluate(y_test, predicted)

'End of epoch 2000. Total Error = 0.09373307280525693'

Accuracy: 82.71%
Confusion Matrix and Accuracy per class:
      Bad  Good  Mid  Accuracy
Bad     2     0   17  0.105263
Good    0    31   34  0.476923
Mid     5    27  364  0.919192
Average accuracy per class: 50.05%


In [157]:
print(accs)

{200: (0.83125, 0.4340033261085893), 400: (0.84375, 0.5033506454559086), 800: (0.8395833333333333, 0.5226557068662331), 1000: (0.825, 0.46961245382298017), 2000: (0.8270833333333333, 0.5004593846699109)}


In [160]:
epochs_accs = pd.DataFrame(accs)
epochs_accs.index = ["Total accuracy", "Accuracy per class"]
print(epochs_accs)

                        200       400       800       1000      2000
Total accuracy      0.831250  0.843750  0.839583  0.825000  0.827083
Accuracy per class  0.434003  0.503351  0.522656  0.469612  0.500459


Repare que embora a Loss Function decresça para 1000 e 2000 epochs, ao avaliar o modelo no conjunto de testes, há uma queda na acurácia! isso ilustra a ocorrência de um overfitting do modelo. Portanto, nas próximas etapas, iremos tomar 800 epochs como o máximo de iterações durante a fase de treinamento, uma vez que ela mantém um bom nível de acurácia total, equanto eleva a acurácia por classe.

In [163]:
best_epochs = 800

# Learning Rate and Momentum

Fixando o número de epochs em 500 e utilizando a arquitura de rede com melhor desemenho (2 camadas intermediárias: 5 neurônios na primeira e 11 na segunda), vamos variar os parâmetros learning rate (eta) e momentum (alfa), e vamos observer como eles interferem no aprendizado.

## Learning Rate = 0.3 e Momentum = 0.3

In [190]:
accs = dict()

In [191]:
eta = 0.3
alpha = 0.3

random.seed(0)
best_layers = [Layer(best_N1, N), Layer(best_N2, best_N1), Layer(n_classes, best_N2)]
mlp = MLP.MLPClassifier(*best_layers)
mlp.train(X_train, y_train, eta=eta, alpha=alpha, tol=1e-4, epoch_max=best_epochs, 
          print_status=False, shuffle=True)

predicted = mlp.predict(X_test)
accs[(eta,alpha)] = evaluate(y_test, predicted)

'End of epoch 800. Total Error = 0.12757128237951523'

Accuracy: 83.75%
Confusion Matrix and Accuracy per class:
      Bad  Good  Mid  Accuracy
Bad     1     0   18  0.052632
Good    0    26   39  0.400000
Mid     2    19  375  0.946970
Average accuracy per class: 46.65%


## Learning Rate = 0.3 e Momentum = 0.5

In [ ]:
eta = 0.3
alpha = 0.5

random.seed(0)
best_layers = [Layer(best_N1, N), Layer(best_N2, best_N1), Layer(n_classes, best_N2)]
mlp = MLP.MLPClassifier(*best_layers)
mlp.train(X_train, y_train, eta=eta, alpha=alpha, tol=1e-4, epoch_max=best_epochs, 
          print_status=False, shuffle=True)

predicted = mlp.predict(X_test)
accs[(eta,alpha)] = evaluate(y_test, predicted)

'End of epoch 34. Total Error = 0.21697130441251308'

## Learning Rate = 0.3 e Momentum = 0.8

In [ ]:
eta = 0.3
alpha = 0.8

random.seed(0)
best_layers = [Layer(best_N1, N), Layer(best_N2, best_N1), Layer(n_classes, best_N2)]
mlp = MLP.MLPClassifier(*best_layers)
mlp.train(X_train, y_train, eta=eta, alpha=alpha, tol=1e-4, epoch_max=best_epochs, 
          print_status=False, shuffle=True)

predicted = mlp.predict(X_test)
accs[(eta,alpha)] = evaluate(y_test, predicted)

## Learning Rate = 0.5 e Momentum = 0.3

In [ ]:
eta = 0.5
alpha = 0.3

random.seed(0)
best_layers = [Layer(best_N1, N), Layer(best_N2, best_N1), Layer(n_classes, best_N2)]
mlp = MLP.MLPClassifier(*best_layers)
mlp.train(X_train, y_train, eta=eta, alpha=alpha, tol=1e-4, epoch_max=best_epochs, 
          print_status=False, shuffle=True)

predicted = mlp.predict(X_test)
accs[(eta,alpha)] = evaluate(y_test, predicted)

## Learning Rate = 0.5 e Momentum = 0.5

In [ ]:
eta = 0.5
alpha = 0.5

random.seed(0)
best_layers = [Layer(best_N1, N), Layer(best_N2, best_N1), Layer(n_classes, best_N2)]
mlp = MLP.MLPClassifier(*best_layers)
mlp.train(X_train, y_train, eta=eta, alpha=alpha, tol=1e-4, epoch_max=best_epochs, 
          print_status=False, shuffle=True)

predicted = mlp.predict(X_test)
accs[(eta,alpha)] = evaluate(y_test, predicted)

## Learning Rate = 0.5 e Momentum = 0.8

In [ ]:
eta = 0.5
alpha = 0.8

random.seed(0)
best_layers = [Layer(best_N1, N), Layer(best_N2, best_N1), Layer(n_classes, best_N2)]
mlp = MLP.MLPClassifier(*best_layers)
mlp.train(X_train, y_train, eta=eta, alpha=alpha, tol=1e-4, epoch_max=best_epochs, 
          print_status=False, shuffle=True)

predicted = mlp.predict(X_test)
accs[(eta,alpha)] = evaluate(y_test, predicted)

## Learning Rate = 0.8 e Momentum = 0.3

In [ ]:
eta = 0.8
alpha = 0.3

random.seed(0)
best_layers = [Layer(best_N1, N), Layer(best_N2, best_N1), Layer(n_classes, best_N2)]
mlp = MLP.MLPClassifier(*best_layers)
mlp.train(X_train, y_train, eta=eta, alpha=alpha, tol=1e-4, epoch_max=best_epochs, 
          print_status=False, shuffle=True)

predicted = mlp.predict(X_test)
accs[(eta,alpha)] = evaluate(y_test, predicted)

## Learning Rate = 0.8 e Momentum = 0.5

In [ ]:
eta = 0.8
alpha = 0.5

random.seed(0)
best_layers = [Layer(best_N1, N), Layer(best_N2, best_N1), Layer(n_classes, best_N2)]
mlp = MLP.MLPClassifier(*best_layers)
mlp.train(X_train, y_train, eta=eta, alpha=alpha, tol=1e-4, epoch_max=best_epochs, 
          print_status=False, shuffle=True)

predicted = mlp.predict(X_test)
accs[(eta,alpha)] = evaluate(y_test, predicted)

## Learning Rate = 0.8 e Momentum = 0.8

In [ ]:
eta = 0.8
alpha = 0.8

random.seed(0)
best_layers = [Layer(best_N1, N), Layer(best_N2, best_N1), Layer(n_classes, best_N2)]
mlp = MLP.MLPClassifier(*best_layers)
mlp.train(X_train, y_train, eta=eta, alpha=alpha, tol=1e-4, epoch_max=best_epochs, 
          print_status=False, shuffle=True)

predicted = mlp.predict(X_test)
accs[(eta,alpha)] = evaluate(y_test, predicted)

## Learning Rate = 1 e Momentum = 1

In [ ]:
eta = 1
alpha = 1

random.seed(0)
best_layers = [Layer(best_N1, N), Layer(best_N2, best_N1), Layer(n_classes, best_N2)]
mlp = MLP.MLPClassifier(*best_layers)
mlp.train(X_train, y_train, eta=eta, alpha=alpha, tol=1e-4, epoch_max=best_epochs, 
          print_status=False, shuffle=True)

predicted = mlp.predict(X_test)
accs[(eta,alpha)] = evaluate(y_test, predicted)

In [ ]:
accs

In [ ]:
for k, v in accs.values():
    print(k)
    print(v)

In [186]:
total_acc = pd.DataFrame()
avg_acc = pd.DataFrame()

for run, accs in accs:
    total_acc.at[str(run[0]), str(run[1])] = accs[0]
    avg_acc.at[str(run[0]), str(run[1])] = accs[1]
    
print(total_acc)

TypeError: 'float' object is not iterable

In [ ]:
print(avg_acc)

Podemos observar que, quando o learning rate e momentum são de magnitude elevada (próximos à 1)

# Training and Test Sets

Agora, vamos variar o tamanho dos conjuntos de treinamento e teste utilizando a melhor arquitetura encontrada acima e os melhores valores de learning rate e momentum.

# Better Pre Processing Data

Como podemos observar nas classificações acima, na base de dados considerada, há poucos exemplos da classe Bad. Como consequência, como estamos dando o mesmo peso para um erro cometido em qualquer classe, a rede acaba por classificar a maioria dos exemplos como sendo pertecentes à classe com maior número de exemplos ('Mid').

A seguir, vamos considerar uma base de dados com um número balanceado entre as classes. Para tal, vamos tomar o tamanho da menor classe e escolher exemplos aleatórios das demais classes para igualar esse número.

Também vamos considerar uma base de dados com exemplos artificiais que serão criados para igualar o número de exemplos da menor classe com o número de exemplos da classe intermediária. Não igualaremos o número de exemplares pela classe de maior cardinalidade, uma vez que, no mundo real, é normal que haja mais itens de qualidade intermediária que itens de qualidade ruim ou boa. 

Para melhor comparar as base de dados e não ser influenciado pela aleatoriedade com a qual os conjuntos de treinamento e teste são escolhidos, iremos avaliar os modelos utilizando CrossValidation Stratified com 10 folds!

## Under Sampling

## Over Sampling

## Original Data

# Regressão